# Start Daemons
* reduce cold start latency by running servers

In [ ]:
import os
import subprocess
import time

In [ ]:
procs = []

In [ ]:
def runcmd(cmdlist, env=None):
    procs.append(
        subprocess.Popen(
            cmdlist, env=env, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT
        )
    )

In [ ]:
sources = [
    "mal",
    "anilist",
    "kitsu",
    "animeplanet",
]

In [ ]:
for i, source in enumerate(sources):
    env = os.environ.copy()
    env["RSYS_LIST_SOURCE"] = source
    runcmd(
        [
            "gunicorn",
            "-w",
            "2",
            "FetchMediaLists:app",
            "-b",
            f"localhost:{3020+i}",
        ],
        env,
    )

In [ ]:
runcmd(["julia", "-t", "4", "Recommendations.jl", "3030"])

In [ ]:
os.chdir("../InferenceAlphas")

In [ ]:
for i in range(8):
    runcmd(
        [
            "julia",
            "--startup-file=no",
            "-t",
            "auto",
            "-e",
            f"using DaemonMode; serve({3000+i})",
        ]
    )

In [ ]:
for server, workers, port in zip(["Transformer", "BagOfWords"], [2, 8], [3010, 3011]):
    runcmd(["gunicorn", "-w", str(workers), f"{server}:app", "-b", f"localhost:{port}"])

In [ ]:
# keep the notebook alive to keep the procs alive
while True:
    time.sleep(3600)

In [ ]:
for p in procs:
    p.terminate()